# Drop the Losers Experiment

https://www.ncbi.nlm.nih.gov/pubmed/16053251

https://onlinelibrary.wiley.com/doi/epdf/10.1002/bimj.200410119

https://www.oatext.com/pdf/CRT-3-186.pdf

https://searchworks.stanford.edu/articles/edo__ejs22481119

## Step 1

Let $k$ be the number of "treatments." Generate $Y_1, \dots, Y_k$, where $Y_i \in \mathbb{R}^{n_i}$ for any natural $n_i$. Think of the vectors $Y_i$ as lists of observations for each treatment $i \in \{1, \dots, k\}$.

Define $Y_i \sim N(\mu_i, \sigma^2)$ (*note $\sigma^2$ is constant across all classes*).

Define $i^* = \arg\max(\bar{Y}_i)$.

In [1]:
#NULL cell where data is loaded

In [1]:
mytest = abs(mean(stage1[['X1']])) > 2 / sqrt(nrow(stage1))

ERROR: Error in mean(stage1[["X1"]]): object 'stage1' not found


## Step 2

Let $m$ be the sample size for an additional experiment on the optimal treatment $i^*$.

Define $W \sim N(\mu_{i^*}, \sigma^2)$, where $W \in \mathbb{R}^m$.

Define $\mu_{i^*} = \frac{n_{i^*} \bar{Y}_{i^*} + \bar{W}}{n_{i^*} + m}$

In [ ]:
compute_sufficient_statistics <- function(selection) {
    suff_stat = sum(stage1[['X1']])
    names(suff_stat) = 'X1'
    return(as.data.frame(t(suff_stat)))
}

compute_estimators <- function(selection) {
    suff_stat = compute_sufficient_statistics(selection)
    value = suff_stat[['X1']] / nrow(stage1)
    names(value) = 'X1'
    indicator = function(sim_selection) {
        return(sim_selection[['mytest']] == TRUE)
    }
    return(list(X1=list(value=value, identifier='X1', indicator=indicator)))
}

estimate_var <- function(selection) {
    variance = var(c(stage1[['X1']]))
    cross = variance
    names(cross) = 'X1'
    names(variance) = 'X1'
    return(list(X1=list(identifier='X1', var=variance / nrow(stage1),
                            cross=cross)))
}


resample_data <- function(data, selection) {
    stage1 = data[["stage1"]]
    stage2 = data[["stage2"]]
    
    vars = apply(stage1, 2, var)
    
    means = apply(stage1, 2, mean)
    
    stage1_sim = stage1
    stage1_sim[['X1']] = rnorm(nrow(stage1)) * sqrt(vars[['X1']]) * (0.2 + 1.6 * runif(1)) + means[['X1']]
    stage2_sim = stage2
    
    sim_env = new.env()
    assign('stage1', as.data.frame(stage1_sim), env=sim_env)
    assign('stage2', as.data.frame(stage2_sim), env=sim_env)
    return(sim_env)
    
}

